In [1]:
!pip install feedparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install deep_translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
import feedparser
import csv
import pandas as pd
import re
from sklearn.metrics import make_scorer

In [64]:
from pymystem3 import Mystem
import re
from textblob import TextBlob, Word
import nltk
import pymorphy2
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from deep_translator import GoogleTranslator
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from tqdm import notebook
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight

##### Укажите словарь с RSS-лентой, обратите внимание, что не все RSS-ленты имеют доступный для парсинга формат, проверяйте
##### Укажите два вектора по которым будет производить поиск, символ '|' означает "или", если нужен только один вектор, в конце другого добавьте '|' - это будет означать, что вектор ищет все слова. Векторы не чувствительны к регистрам
##### Укажите два пути для файлов, для всех новостей и для определенных новостей

In [9]:
newsurls = {'Kommersant': 'https://www.kommersant.ru/RSS/news.xml',
'Lenta.ru': 'https://lenta.ru/rss/',
'Vesti': 'https://www.vesti.ru/vesti.rss'} #пример словаря RSS-лент 
                                           #русскоязычных источников

f_all_news = 'allnews23march.csv'
f_certain_news = 'certainnews23march.csv'

vector1 = 'бизнес' #пример таргетов
vector2 = ''

In [10]:
def parseRSS( rss_url ): #функция получает линк на рсс ленту, возвращает распаршенную ленту с помощью feedpaeser
    return feedparser.parse( rss_url )  
    
def getHeadlines( rss_url ): #функция для получения заголовков новости
    headlines = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        headlines.append(newsitem['title'])
    return headlines

def getDescriptions( rss_url ): #функция для получения описания новости
    descriptions = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        descriptions.append(newsitem['description'])
    return descriptions

def getLinks( rss_url ): #функция для получения ссылки на источник новости
    links = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        links.append(newsitem['link'])
    return links

def getDates( rss_url ): #функция для получения даты публикации новости
    dates = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        dates.append(newsitem['published'])
    return dates

In [11]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# Прогоняем нашии URL и добавляем их в наши пустые списки
for key,url in newsurls.items():
    allheadlines.extend( getHeadlines( url ) )
    
for key,url in newsurls.items():
    alldescriptions.extend( getDescriptions( url ) )
    
for key,url in newsurls.items():
    alllinks.extend( getLinks( url ) )
    
for key,url in newsurls.items():
    alldates.extend( getDates( url ) )

In [12]:
def write_all_news(all_news_filepath): #функция для записи всех новостей в .csv, возвращает нам этот датасет
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #функция для поиска, а затем записи
                                                                                         #определенных новостей по таргета,
                                                                                        #затем возвращает этот датасет
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [13]:
table = write_all_news(f_all_news) #.drop(['Description', 'Publication Date'], axis=1) #все новости
table

,Title,Description,Links,Publication Date
0,Командующим войсками спецоперации на Украине н...,Командующим объединенной группировкой войск в ...,https://www.kommersant.ru/doc/5606182,"Sat, 08 Oct 2022 15:13:00 +0300"
1,Керченскую паромную переправу пообещали запуст...,"Оперативный штаб Краснодарского края сообщил, ...",https://www.kommersant.ru/doc/5606179,"Sat, 08 Oct 2022 15:04:00 +0300"
2,На месте паромной переправы в порту «Кавказ» о...,Вице-губернатор Краснодарского края Евгений На...,https://www.kommersant.ru/doc/5606175,"Sat, 08 Oct 2022 14:52:00 +0300"
3,В Воронежской области усилили охрану железнодо...,Власти Воронежской области по поручению губерн...,https://www.kommersant.ru/doc/5606174,"Sat, 08 Oct 2022 14:47:00 +0300"
4,Умер старейший депутат Сахалинской облдумы Вит...,На Сахалине в возрасте 80 лет умер старейший д...,https://www.kommersant.ru/doc/5606144,"Sat, 08 Oct 2022 14:25:00 +0300"
...,...,...,...,...
509,Россияне потеряли интерес к покупке жилья в по...,В сентябре 2022 года спрос со стороны россиян ...,https://lenta.ru/news/2022/10/07/no_florida/,"Fri, 07 Oct 2022 20:24:00 +0300"
510,Путин и Зеленский согласились приехать на самм...,Президент России Владимир Путин и украинский л...,https://lenta.ru/news/2022/10/07/g200/,"Fri, 07 Oct 2022 20:21:00 +0300"
511,Кремль связал слова Зеленского с проведением СВО,Слова президента Украины Владимира Зеленского ...,https://lenta.ru/news/2022/10/07/svoo/,"Fri, 07 Oct 2022 20:17:00 +0300"
512,В «Манчестер Юнайтед» оценили вероятность уход...,В английском клубе «Манчестер Юнайтед» оценили...,https://lenta.ru/news/2022/10/07/ronaldo/,"Fri, 07 Oct 2022 20:16:00 +0300"


In [14]:
def clear_text(text):
    prom = re.sub(r'[^a-zA-Zа-яА-Я]',' ', text).split()
    return ' '.join(prom)


In [15]:
def lemmatize1(text):
    morph = pymorphy2.MorphAnalyzer()
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return " ".join(res)

In [16]:
table['clear'] = table['Title'].apply(clear_text)

In [17]:
table['lemm'] = table['clear'].apply(lemmatize1)

In [18]:
table['clear_text'] = table['Description'].apply(clear_text)

In [19]:
table['lemm_text'] = table['clear_text'].apply(lemmatize1)

In [20]:
table

,Title,Description,Links,Publication Date,clear,lemm,clear_text,lemm_text
0,Командующим войсками спецоперации на Украине н...,Командующим объединенной группировкой войск в ...,https://www.kommersant.ru/doc/5606182,"Sat, 08 Oct 2022 15:13:00 +0300",Командующим войсками спецоперации на Украине н...,командующий войско спецоперация на украина наз...,Командующим объединенной группировкой войск в ...,командующий объединить группировка войско в ра...
1,Керченскую паромную переправу пообещали запуст...,"Оперативный штаб Краснодарского края сообщил, ...",https://www.kommersant.ru/doc/5606179,"Sat, 08 Oct 2022 15:04:00 +0300",Керченскую паромную переправу пообещали запуст...,керченский паромный переправа пообещать запуст...,Оперативный штаб Краснодарского края сообщил ч...,оперативный штаб краснодарский край сообщить ч...
2,На месте паромной переправы в порту «Кавказ» о...,Вице-губернатор Краснодарского края Евгений На...,https://www.kommersant.ru/doc/5606175,"Sat, 08 Oct 2022 14:52:00 +0300",На месте паромной переправы в порту Кавказ обр...,на место паромный переправа в порт кавказ обра...,Вице губернатор Краснодарского края Евгений На...,вица губернатор краснодарский край евгений нау...
3,В Воронежской области усилили охрану железнодо...,Власти Воронежской области по поручению губерн...,https://www.kommersant.ru/doc/5606174,"Sat, 08 Oct 2022 14:47:00 +0300",В Воронежской области усилили охрану железнодо...,в воронежский область усилить охрана железнодо...,Власти Воронежской области по поручению губерн...,власть воронежский область по поручение губерн...
4,Умер старейший депутат Сахалинской облдумы Вит...,На Сахалине в возрасте 80 лет умер старейший д...,https://www.kommersant.ru/doc/5606144,"Sat, 08 Oct 2022 14:25:00 +0300",Умер старейший депутат Сахалинской облдумы Вит...,умереть старый депутат сахалинский облдума вит...,На Сахалине в возрасте лет умер старейший депу...,на сахалин в возраст год умереть старый депута...
...,...,...,...,...,...,...,...,...
509,Россияне потеряли интерес к покупке жилья в по...,В сентябре 2022 года спрос со стороны россиян ...,https://lenta.ru/news/2022/10/07/no_florida/,"Fri, 07 Oct 2022 20:24:00 +0300",Россияне потеряли интерес к покупке жилья в по...,россиянин потерять интерес к покупка жильё в п...,В сентябре года спрос со стороны россиян на пр...,в сентябрь год спрос с сторона россиянин на пр...
510,Путин и Зеленский согласились приехать на самм...,Президент России Владимир Путин и украинский л...,https://lenta.ru/news/2022/10/07/g200/,"Fri, 07 Oct 2022 20:21:00 +0300",Путин и Зеленский согласились приехать на самм...,путин и зеленский согласиться приехать на самм...,Президент России Владимир Путин и украинский л...,президент россия владимир путин и украинский л...
511,Кремль связал слова Зеленского с проведением СВО,Слова президента Украины Владимира Зеленского ...,https://lenta.ru/news/2022/10/07/svoo/,"Fri, 07 Oct 2022 20:17:00 +0300",Кремль связал слова Зеленского с проведением СВО,кремль связать слово зеленский с проведение сво,Слова президента Украины Владимира Зеленского ...,слово президент украина владимир зеленский о п...
512,В «Манчестер Юнайтед» оценили вероятность уход...,В английском клубе «Манчестер Юнайтед» оценили...,https://lenta.ru/news/2022/10/07/ronaldo/,"Fri, 07 Oct 2022 20:16:00 +0300",В Манчестер Юнайтед оценили вероятность ухода ...,в манчестер юнайтед оценить вероятность уход р...,В английском клубе Манчестер Юнайтед оценили в...,в английский клуб манчестер юнайтед оценить ве...


In [21]:
stop_words = set(stopwords.words('russian'))
tf_idf_vect = TfidfVectorizer(stop_words=stop_words)

In [22]:
features = tf_idf_vect.fit_transform(list(table['lemm']))
articles_array = features.toarray()
articles_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [23]:
help_df_for_index = pd.DataFrame(data=articles_array, columns=[str(k) for k in range(len(articles_array[0]))])
help_df_for_index['3'].value_counts()

0.000000    513
0.370452      1
Name: 3, dtype: int64

In [24]:
#Похожие новости
all_distances = [] #массив массивов расстояний
for i in notebook.tqdm(range(len(articles_array))):
  dist = [] #массив расстояний от конкретного текста
  for j in range(len(articles_array)):
    di = distance.euclidean(articles_array[i], articles_array[j])
    dist.append(di)
  all_distances.append(dist)
all_distances = np.array(all_distances)

  0%|          | 0/514 [00:00<?, ?it/s]

In [25]:
all_distances_df = pd.DataFrame(data=all_distances, columns=[str(k) for k in range(len(all_distances[0]))])
all_distances_df

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,512,513
0,0.000000,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,...,1.414214,1.414214,1.414214,1.326976,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214
1,1.414214,0.000000,1.243135,1.414214,1.414214,1.140253,1.414214,1.414214,1.414214,1.414214,...,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214
2,1.414214,1.243135,0.000000,1.414214,1.414214,1.275385,1.414214,1.414214,1.414214,1.414214,...,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214
3,1.414214,1.414214,1.414214,0.000000,1.414214,1.414214,1.250533,1.380814,1.384202,1.414214,...,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214
4,1.414214,1.414214,1.414214,1.414214,0.000000,1.414214,1.414214,1.414214,1.414214,1.414214,...,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,...,1.414214,1.414214,1.414214,1.414214,1.370424,0.000000,1.414214,1.414214,1.414214,1.358659
510,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,...,1.250445,1.414214,1.414214,1.351484,1.414214,1.414214,0.000000,1.329826,1.414214,1.414214
511,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,...,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.329826,0.000000,1.414214,1.414214
512,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,...,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,1.414214,0.000000,1.414214


In [26]:
table['Title'].loc[all_distances_df['86'].sort_values().index[0:3]]

86      В России в сентябре зафиксировали инфляцию 0,05%
310    В конце сентября продажи армейских ботинок и б...
272    Объем частных инвестиций в российские акции в ...
Name: Title, dtype: object

In [27]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) #новости по вектору

,Title,Description,Links,Publication Date
17,Ростуризм призвал владельцев крымских отелей п...,Ростуризм работает с владельцами отелей и сана...,https://www.kommersant.ru/doc/5606132,"Sat, 08 Oct 2022 12:18:00 +0300"
46,В Подмосковье по подозрению в мошенничестве ар...,"Как стало известно «Ъ», Домодедовский суд по х...",https://www.kommersant.ru/doc/5606069,"Sat, 08 Oct 2022 08:46:00 +0300"
72,Обнаружена возможная утечка личных данных боле...,Компания Meta (запрещена в России как экстреми...,https://www.kommersant.ru/doc/5605689,"Fri, 07 Oct 2022 22:31:00 +0300"
115,"Маск назвал себя фанатом Украины, но не третье...",Американский предприниматель Илон Маск продолж...,https://www.kommersant.ru/doc/5605509,"Fri, 07 Oct 2022 14:32:00 +0300"
142,Производитель бумаги SvetoCopy продал российск...,Американская компания Sylvamo Corporation прод...,https://www.kommersant.ru/doc/5596229,"Fri, 07 Oct 2022 11:21:00 +0300"
146,Skillbox: только 5% россиян довольны текущим м...,Из-за недостатка компетенций и «предшествовавш...,https://www.kommersant.ru/doc/5596166,"Fri, 07 Oct 2022 11:01:00 +0300"
203,Credit Suisse ищет инвесторов под выделение ко...,Credit Suisse в рамках готовящегося плана рест...,https://www.kommersant.ru/doc/5595896,"Thu, 06 Oct 2022 20:21:00 +0300"
204,Власти Франции представили план экономии элект...,Правительство Франции представило план «всеобщ...,https://www.kommersant.ru/doc/5595868,"Thu, 06 Oct 2022 20:15:00 +0300"
228,Дерипаска призвал к деполитизации международны...,Российский бизнесмен Олег Дерипаска назвал тек...,https://www.kommersant.ru/doc/5595797,"Thu, 06 Oct 2022 17:14:00 +0300"
232,«Конкорд» пригрозил ответственностью СМИ за со...,"Компания «Конкорд», которая принадлежит Евгени...",https://www.kommersant.ru/doc/5595743,"Thu, 06 Oct 2022 16:16:00 +0300"


Алгоритм - придумываем синонимы, ищем близкие к ним слова в тексте. Берём топ 2-3 статьи

In [28]:
from nltk.corpus import wordnet

synonyms = []

for syn in wordnet.synsets("nigger"):
    for i in syn.lemmas():
        synonyms.append(i.name())

print(set(synonyms))

{'nigga', 'nigra', 'coon', 'nigger', 'jigaboo', 'spade'}


In [29]:
synonyms = []

for syn in wordnet.synsets("nigger"):
    for lm in syn.lemmas():
             synonyms.append(lm.name())

rusynonyms = []

for synonym in synonyms:
    rusynonyms.append(GoogleTranslator(source='en', target='ru').translate(synonym))

print(rusynonyms)

['негр', 'ниггер', 'лопата', 'енот', 'джигабу', 'нигра']


Синонимы для владельца: Налоги, управление, собственник, акция, предприятие,  директор, фондовый рынок, прибыль, убыток, организация

Синонимы для бухгалтера: финансовый учет, отчет, прибыль, убыток, денежные средства, контроль, зарплата, документы, законодательство, документооборот

In [30]:
russian_owner = ['налог', 'управление', 'собственник', 'акция', 'предприятие', 'директор', 'фондовый рынок', 'прибыль', 'убыток', 'организация']
english_owner = []
for rus_word in russian_owner:
      english_owner.append(GoogleTranslator(source='ru', target='en').translate(rus_word))
english_owner

['tax',
 'control',
 'owner',
 'stock',
 'company',
 'director',
 'stock market',
 'profit',
 'lesion',
 'organization']

In [31]:
russian_accountant = ['финансовый учет', 'отчет', 'прибыль', 'убыток', 'денежные средства', 'контроль', 'зарплата', 'документы', 'законодательство', 'документооборот']
english_accountant = []
for rus_word in russian_accountant:
      english_accountant.append(GoogleTranslator(source='ru', target='en').translate(rus_word))
english_accountant

['financial Accounting',
 'report',
 'profit',
 'lesion',
 'cash',
 'control',
 'salary',
 'the documents',
 'legislation',
 'document flow']

In [32]:
#синонимы для гендира
russian_owner_synonyms = []
for word in english_owner:  
  for syn in wordnet.synsets(word):
      for lm in syn.lemmas():
             russian_owner_synonyms.append(lm.name())

rusynonyms_owner = []

for synonym in russian_owner_synonyms:
    rusynonyms_owner.append(GoogleTranslator(source='en', target='ru').translate(synonym))
list(set(rusynonyms_owner))

['задача',
 'прибыль',
 'заработок',
 'сопровождать',
 'нести',
 'stock_up',
 'росток',
 'keep_company',
 'хранить',
 'родословная',
 'ферма_животное',
 'оценивать',
 'установить',
 'театральный_директор',
 'избитый',
 'control_condition',
 'Чистая прибыль',
 'оборот_а_прибыль',
 'владелец',
 'управление',
 'общение',
 'stock_certificate',
 'выгода',
 'администрация',
 'правление',
 'управляющий делами',
 'Компания',
 'спуск',
 'напряжение',
 'левкои',
 'ensure',
 'обыденность',
 'изношенный',
 'застраховать',
 'формирование',
 'видеть',
 'Старая шляпа',
 'ранить',
 'домашний скот',
 'бордюр',
 'учреждение',
 'запас',
 'Hold_in',
 'чистая_прибыль',
 'банальный',
 'контроллер',
 'фонд',
 'корабельная_компания',
 'товарищество',
 'каудекс',
 'корысть',
 'проверять',
 'организация',
 'доход_повышение',
 'режиссер фильма',
 'контроль',
 'налогообложение',
 'порода',
 'линия',
 'гарантировать',
 'директор',
 'видеть_к_этому',
 'мастерство',
 'кровная_линия',
 'line_of_descent',
 'музыкальны

In [33]:
#синонимы для бухгалтера
russian_accountant_synonyms = []
for word in english_accountant:  
  for syn in wordnet.synsets(word):
      for lm in syn.lemmas():
             russian_accountant_synonyms.append(lm.name())

rusynonyms_accountant = []

for synonym in russian_accountant_synonyms:
    rusynonyms_accountant.append(GoogleTranslator(source='en', target='ru').translate(synonym))
list(set(rusynonyms_accountant))

['вознаграждение',
 'господство',
 'история',
 'платить',
 'Наличные',
 'выгода',
 'прибыль',
 'заработок',
 'законодательный',
 'контроль',
 'проверять',
 'hard_cash',
 'отчет',
 'содержать',
 'ensure',
 'гарантировать',
 'немедленная оплата',
 'репутация',
 'застраховать',
 'поражение',
 'письменный отчет',
 'покрытие',
 'мастерство',
 'видеть',
 'репортаж',
 'ранить',
 'команда',
 'сеть',
 'записать',
 'действовать',
 'мастер',
 'сдержанность',
 'бордюр',
 'keep_in_line',
 'сочинение',
 'Джон_Кэш',
 'исследование',
 'держать',
 'учетная запись',
 'описывать',
 'законотворчество',
 'тема',
 'Hold_in',
 'умеренный',
 'видеть_к_этому',
 'законодательство',
 'чистая_прибыль',
 'Проверьте',
 'установить',
 'восхождение',
 'report_card',
 'контроллер',
 'Джонни Кэш',
 'заработная плата',
 'cash_in',
 'манипулировать',
 'прирост',
 'зарплата',
 'закон_закон',
 'твердая валюта',
 'control_condition',
 'Чистая прибыль',
 'оборот_а_прибыль',
 'бумага',
 'корысть',
 'наличные']

In [34]:
GoogleTranslator(source='en', target='ru').translate(" ".join(english_accountant))

'финансовый бухгалтерский отчет прибыль убытки кассовый контроль заработная плата документы законодательство документооборот'

In [35]:
" ".join(english_accountant)

'financial Accounting report profit lesion cash control salary the documents legislation document flow'

In [36]:
def markup(text, owner_sheet, accountant_sheet):
  text = text.split()
  owner_counter = 0
  accountant_counter = 0
  for i in range(len(text)):
    if text[i] in owner_sheet:
      owner_counter+=1
    if text[i] in accountant_sheet:
      accountant_counter+=1
  if owner_counter > accountant_counter and owner_counter != 0 and accountant_counter != 0:
    return 1
  elif accountant_counter >= owner_counter and owner_counter != 0 and accountant_counter != 0:
    return 0
  else:
    return 2

In [37]:
table['target'] = table['lemm_text'].apply(markup, args=(list(set(rusynonyms_owner)), list(set(rusynonyms_accountant)), ))

In [38]:
table['target'].value_counts()

2    423
0     54
1     37
Name: target, dtype: int64

In [39]:
table.loc[:,['lemm_text', 'target']]

,lemm_text,target
0,командующий объединить группировка войско в ра...,2
1,оперативный штаб краснодарский край сообщить ч...,2
2,вица губернатор краснодарский край евгений нау...,2
3,власть воронежский область по поручение губерн...,2
4,на сахалин в возраст год умереть старый депута...,2
...,...,...
509,в сентябрь год спрос с сторона россиянин на пр...,2
510,президент россия владимир путин и украинский л...,2
511,слово президент украина владимир зеленский о п...,2
512,в английский клуб манчестер юнайтед оценить ве...,0


In [40]:
 df = pd.read_csv('/content/pupupupupu_novosti.csv')
 df

,Title,Description,Publication Date
0,Минобороны считает преждевременной и ненужной ...,Минобороны РФ отреагировало на отмену в некото...,"Fri, 07 Oct 2022 23:37:00 +0300"
1,FT: Украина сообщила о перебоях в работе Starlink,"Вооруженные силы Украины заявили, что в работе...","Fri, 07 Oct 2022 23:04:00 +0300"
2,"Зеленский заявил, что под «превентивными удара...",Президент Украины Владимир Зеленский заявил BB...,"Fri, 07 Oct 2022 22:54:00 +0300"
3,Обнаружена возможная утечка личных данных боле...,Компания Meta (запрещена в России как экстреми...,"Fri, 07 Oct 2022 22:31:00 +0300"
4,В Курской области вновь обстреляли поселок,Курский губернатор Роман Старовойт сообщил об ...,"Fri, 07 Oct 2022 22:02:00 +0300"
...,...,...,...
908,Минобороны признало наличие специальных спутни...,4 октября Минобороны России проинформировало п...,"Tue, 04 Oct 2022 20:15:22 +0300"
909,Влиятельный американский экономист обвинил США...,На Западе продолжается дискуссия по поводу тог...,"Tue, 04 Oct 2022 19:04:08 +0300"
910,"Военный эксперт рассказал, как российской арми...",Наступательные действия ВСУ на Херсонщине и Ха...,"Tue, 04 Oct 2022 17:31:49 +0300"
911,Подсчитаны потери ВСУ в ходе наступления на Ли...,В начале октября ВСУ провели наступление на Ли...,"Tue, 04 Oct 2022 13:48:05 +0300"


In [41]:
df = df.dropna().reset_index(drop=True)
df

,Title,Description,Publication Date
0,Минобороны считает преждевременной и ненужной ...,Минобороны РФ отреагировало на отмену в некото...,"Fri, 07 Oct 2022 23:37:00 +0300"
1,FT: Украина сообщила о перебоях в работе Starlink,"Вооруженные силы Украины заявили, что в работе...","Fri, 07 Oct 2022 23:04:00 +0300"
2,"Зеленский заявил, что под «превентивными удара...",Президент Украины Владимир Зеленский заявил BB...,"Fri, 07 Oct 2022 22:54:00 +0300"
3,Обнаружена возможная утечка личных данных боле...,Компания Meta (запрещена в России как экстреми...,"Fri, 07 Oct 2022 22:31:00 +0300"
4,В Курской области вновь обстреляли поселок,Курский губернатор Роман Старовойт сообщил об ...,"Fri, 07 Oct 2022 22:02:00 +0300"
...,...,...,...
808,Минобороны признало наличие специальных спутни...,4 октября Минобороны России проинформировало п...,"Tue, 04 Oct 2022 20:15:22 +0300"
809,Влиятельный американский экономист обвинил США...,На Западе продолжается дискуссия по поводу тог...,"Tue, 04 Oct 2022 19:04:08 +0300"
810,"Военный эксперт рассказал, как российской арми...",Наступательные действия ВСУ на Херсонщине и Ха...,"Tue, 04 Oct 2022 17:31:49 +0300"
811,Подсчитаны потери ВСУ в ходе наступления на Ли...,В начале октября ВСУ провели наступление на Ли...,"Tue, 04 Oct 2022 13:48:05 +0300"


In [42]:
df['clear_text'] = df['Description'].apply(clear_text)

In [43]:
df['lemm_text'] = df['clear_text'].apply(lemmatize1)

In [44]:
df

,Title,Description,Publication Date,clear_text,lemm_text
0,Минобороны считает преждевременной и ненужной ...,Минобороны РФ отреагировало на отмену в некото...,"Fri, 07 Oct 2022 23:37:00 +0300",Минобороны РФ отреагировало на отмену в некото...,минобороны рф отреагировать на отмена в некото...
1,FT: Украина сообщила о перебоях в работе Starlink,"Вооруженные силы Украины заявили, что в работе...","Fri, 07 Oct 2022 23:04:00 +0300",Вооруженные силы Украины заявили что в работе ...,вооружённый сила украина заявить что в работа ...
2,"Зеленский заявил, что под «превентивными удара...",Президент Украины Владимир Зеленский заявил BB...,"Fri, 07 Oct 2022 22:54:00 +0300",Президент Украины Владимир Зеленский заявил BB...,президент украина владимир зеленский заявить b...
3,Обнаружена возможная утечка личных данных боле...,Компания Meta (запрещена в России как экстреми...,"Fri, 07 Oct 2022 22:31:00 +0300",Компания Meta запрещена в России как экстремис...,компания meta запретить в россия как экстремис...
4,В Курской области вновь обстреляли поселок,Курский губернатор Роман Старовойт сообщил об ...,"Fri, 07 Oct 2022 22:02:00 +0300",Курский губернатор Роман Старовойт сообщил об ...,курский губернатор роман старовойт сообщить о ...
...,...,...,...,...,...
808,Минобороны признало наличие специальных спутни...,4 октября Минобороны России проинформировало п...,"Tue, 04 Oct 2022 20:15:22 +0300",октября Минобороны России проинформировало пре...,октябрь минобороны россия проинформировать пре...
809,Влиятельный американский экономист обвинил США...,На Западе продолжается дискуссия по поводу тог...,"Tue, 04 Oct 2022 19:04:08 +0300",На Западе продолжается дискуссия по поводу тог...,на запад продолжаться дискуссия по повод тот к...
810,"Военный эксперт рассказал, как российской арми...",Наступательные действия ВСУ на Херсонщине и Ха...,"Tue, 04 Oct 2022 17:31:49 +0300",Наступательные действия ВСУ на Херсонщине и Ха...,наступательный действие всу на херсонщина и ха...
811,Подсчитаны потери ВСУ в ходе наступления на Ли...,В начале октября ВСУ провели наступление на Ли...,"Tue, 04 Oct 2022 13:48:05 +0300",В начале октября ВСУ провели наступление на Ли...,в начало октябрь всу провести наступление на л...


In [45]:
df['target'] = df['lemm_text'].apply(markup, args=(list(set(rusynonyms_owner)), list(set(rusynonyms_accountant)), ))
df

,Title,Description,Publication Date,clear_text,lemm_text,target
0,Минобороны считает преждевременной и ненужной ...,Минобороны РФ отреагировало на отмену в некото...,"Fri, 07 Oct 2022 23:37:00 +0300",Минобороны РФ отреагировало на отмену в некото...,минобороны рф отреагировать на отмена в некото...,2
1,FT: Украина сообщила о перебоях в работе Starlink,"Вооруженные силы Украины заявили, что в работе...","Fri, 07 Oct 2022 23:04:00 +0300",Вооруженные силы Украины заявили что в работе ...,вооружённый сила украина заявить что в работа ...,1
2,"Зеленский заявил, что под «превентивными удара...",Президент Украины Владимир Зеленский заявил BB...,"Fri, 07 Oct 2022 22:54:00 +0300",Президент Украины Владимир Зеленский заявил BB...,президент украина владимир зеленский заявить b...,2
3,Обнаружена возможная утечка личных данных боле...,Компания Meta (запрещена в России как экстреми...,"Fri, 07 Oct 2022 22:31:00 +0300",Компания Meta запрещена в России как экстремис...,компания meta запретить в россия как экстремис...,1
4,В Курской области вновь обстреляли поселок,Курский губернатор Роман Старовойт сообщил об ...,"Fri, 07 Oct 2022 22:02:00 +0300",Курский губернатор Роман Старовойт сообщил об ...,курский губернатор роман старовойт сообщить о ...,0
...,...,...,...,...,...,...
808,Минобороны признало наличие специальных спутни...,4 октября Минобороны России проинформировало п...,"Tue, 04 Oct 2022 20:15:22 +0300",октября Минобороны России проинформировало пре...,октябрь минобороны россия проинформировать пре...,1
809,Влиятельный американский экономист обвинил США...,На Западе продолжается дискуссия по поводу тог...,"Tue, 04 Oct 2022 19:04:08 +0300",На Западе продолжается дискуссия по поводу тог...,на запад продолжаться дискуссия по повод тот к...,2
810,"Военный эксперт рассказал, как российской арми...",Наступательные действия ВСУ на Херсонщине и Ха...,"Tue, 04 Oct 2022 17:31:49 +0300",Наступательные действия ВСУ на Херсонщине и Ха...,наступательный действие всу на херсонщина и ха...,2
811,Подсчитаны потери ВСУ в ходе наступления на Ли...,В начале октября ВСУ провели наступление на Ли...,"Tue, 04 Oct 2022 13:48:05 +0300",В начале октября ВСУ провели наступление на Ли...,в начало октябрь всу провести наступление на л...,2


In [46]:
df['target'].value_counts()

2    683
0     85
1     45
Name: target, dtype: int64

In [47]:
prom_df = df.loc[:, ['lemm_text', 'target']]
prom_df

,lemm_text,target
0,минобороны рф отреагировать на отмена в некото...,2
1,вооружённый сила украина заявить что в работа ...,1
2,президент украина владимир зеленский заявить b...,2
3,компания meta запретить в россия как экстремис...,1
4,курский губернатор роман старовойт сообщить о ...,0
...,...,...
808,октябрь минобороны россия проинформировать пре...,1
809,на запад продолжаться дискуссия по повод тот к...,2
810,наступательный действие всу на херсонщина и ха...,2
811,в начало октябрь всу провести наступление на л...,2


In [48]:
prom_table = table.loc[:, ['lemm_text', 'target']]
prom_table

,lemm_text,target
0,командующий объединить группировка войско в ра...,2
1,оперативный штаб краснодарский край сообщить ч...,2
2,вица губернатор краснодарский край евгений нау...,2
3,власть воронежский область по поручение губерн...,2
4,на сахалин в возраст год умереть старый депута...,2
...,...,...
509,в сентябрь год спрос с сторона россиянин на пр...,2
510,президент россия владимир путин и украинский л...,2
511,слово президент украина владимир зеленский о п...,2
512,в английский клуб манчестер юнайтед оценить ве...,0


In [49]:
full_table = pd.concat([prom_df, prom_table], ignore_index=True)
full_table = full_table.drop_duplicates().reset_index(drop=True)
full_table

,lemm_text,target
0,минобороны рф отреагировать на отмена в некото...,2
1,вооружённый сила украина заявить что в работа ...,1
2,президент украина владимир зеленский заявить b...,2
3,компания meta запретить в россия как экстремис...,1
4,курский губернатор роман старовойт сообщить о ...,0
...,...,...
1022,в правило дорожный движение пдд появиться секц...,2
1023,microsoft назвать способ ускорить работа windo...,2
1024,российский боец смешанный стиль mma магомед ис...,2
1025,канцлер фрг олафа шольц оказаться под давление...,2


In [50]:
full_table['target'].value_counts()

2    870
0    102
1     55
Name: target, dtype: int64

In [51]:
stop_words = set(stopwords.words('russian'))
tf_idf_vect = TfidfVectorizer(stop_words=stop_words)

In [52]:
features_res = tf_idf_vect.fit_transform(list(full_table['lemm_text']))

Тест логистической регрессии для определения новости интересной владельцу бизнеса

In [53]:
def for_owner(value):
  if value == 1:
    return 1
  else:
    return 0

In [54]:
owner_target = full_table['target'].apply(for_owner)
owner_target

0       0
1       1
2       0
3       1
4       0
       ..
1022    0
1023    0
1024    0
1025    0
1026    0
Name: target, Length: 1027, dtype: int64

In [55]:
features_train, features_test, target_train, target_test = train_test_split(
    features_res, owner_target, random_state=12345, test_size=0.2)

In [56]:
owner_regr = LogisticRegression(class_weight='balanced')
owner_regr.fit(features_train, target_train)
predictions_owner_regr = owner_regr.predict(features_test)
f1 = f1_score(target_test, predictions_owner_regr)
f1

0.0

Тест CatBoostClassifier для определения новости интересной бухгалтеру

In [57]:
def for_accountant(value):
  if value == 0:
    return 1
  else:
    return 0

In [58]:
accountant_target = full_table['target'].apply(for_accountant)
accountant_target.value_counts()

0    925
1    102
Name: target, dtype: int64

In [59]:
classes = np.unique(accountant_target)
weights = compute_class_weight(class_weight='balanced', classes=classes, y = accountant_target)
class_weights = dict(zip(classes, weights))

In [60]:
features_train, features_test, target_train, target_test = train_test_split(
    features_res, accountant_target, random_state=12345, test_size=0.2)

In [61]:
clf = CatBoostClassifier(loss_function='MultiClassOneVsAll', class_weights=class_weights)
clf.fit(features_train, target_train)

0:	learn: 0.6901293	total: 226ms	remaining: 3m 45s
1:	learn: 0.6870279	total: 315ms	remaining: 2m 37s
2:	learn: 0.6849865	total: 406ms	remaining: 2m 14s
3:	learn: 0.6789539	total: 514ms	remaining: 2m 7s
4:	learn: 0.6763030	total: 622ms	remaining: 2m 3s
5:	learn: 0.6714716	total: 724ms	remaining: 2m
6:	learn: 0.6676132	total: 821ms	remaining: 1m 56s
7:	learn: 0.6644470	total: 919ms	remaining: 1m 54s
8:	learn: 0.6608803	total: 1.02s	remaining: 1m 52s
9:	learn: 0.6566599	total: 1.11s	remaining: 1m 49s
10:	learn: 0.6537513	total: 1.21s	remaining: 1m 48s
11:	learn: 0.6499556	total: 1.31s	remaining: 1m 47s
12:	learn: 0.6479738	total: 1.4s	remaining: 1m 46s
13:	learn: 0.6456045	total: 1.5s	remaining: 1m 45s
14:	learn: 0.6393478	total: 1.61s	remaining: 1m 45s
15:	learn: 0.6369496	total: 1.7s	remaining: 1m 44s
16:	learn: 0.6345386	total: 1.8s	remaining: 1m 43s
17:	learn: 0.6319729	total: 1.89s	remaining: 1m 43s
18:	learn: 0.6295620	total: 1.99s	remaining: 1m 42s
19:	learn: 0.6260598	total: 2.1s

In [62]:
pr = clf.predict(features_test)
f1 = f1_score(target_test, pr)
f1

0.4999999999999999

In [63]:
target_train.value_counts()

0    744
1     77
Name: target, dtype: int64

Тест для catboost  с макроусреднением

In [88]:
def f1_scorrer(y_true, y_pred):
  return f1_score(y_true, y_pred, average='macro')
f1_scorer = make_scorer(f1_scorrer)

In [77]:
features_train, features_test, target_train, target_test = train_test_split(
    features_res, full_table['target'], random_state=12345, test_size=0.2)

In [78]:
target_train.value_counts()

2    704
0     77
1     40
Name: target, dtype: int64

In [79]:
classes = np.unique(target_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y = target_train)
class_weights = dict(zip(classes, weights))

In [83]:
clf1 = CatBoostClassifier(loss_function='MultiClassOneVsAll', class_weights=class_weights)
parametres = {
              'iterations': range(500, 1100, 200)}
grid = GridSearchCV(clf1, parametres, cv=3, scoring=f1_scorer)
grid.fit(features_res, full_table['target'])

Выходные данные были обрезаны до нескольких последних строк (5000).
0:	learn: 0.6894976	total: 175ms	remaining: 2m 2s
1:	learn: 0.6854434	total: 355ms	remaining: 2m 3s
2:	learn: 0.6822632	total: 535ms	remaining: 2m 4s
3:	learn: 0.6789437	total: 708ms	remaining: 2m 3s
4:	learn: 0.6759354	total: 892ms	remaining: 2m 3s
5:	learn: 0.6733235	total: 1.06s	remaining: 2m 2s
6:	learn: 0.6707591	total: 1.25s	remaining: 2m 4s
7:	learn: 0.6680264	total: 1.42s	remaining: 2m 3s
8:	learn: 0.6652489	total: 1.61s	remaining: 2m 3s
9:	learn: 0.6623224	total: 1.78s	remaining: 2m 3s
10:	learn: 0.6595219	total: 1.98s	remaining: 2m 3s
11:	learn: 0.6572974	total: 2.16s	remaining: 2m 3s
12:	learn: 0.6550328	total: 2.35s	remaining: 2m 4s
13:	learn: 0.6528586	total: 2.54s	remaining: 2m 4s
14:	learn: 0.6508469	total: 2.73s	remaining: 2m 4s
15:	learn: 0.6485860	total: 2.91s	remaining: 2m 4s
16:	learn: 0.6458275	total: 3.09s	remaining: 2m 4s
17:	learn: 0.6441712	total: 3.27s	remaining: 2m 4s
18:	learn: 0.6408778	tot

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f575dd16a90>,
             param_grid={'iterations': range(500, 1100, 200)},
             scoring=make_scorer(f1_scorer))

In [85]:
grid.best_score_

0.5805997046948742

In [86]:
clf = CatBoostClassifier(random_state=12345)
clf.fit(features_train, target_train)

Learning rate set to 0.078335
0:	learn: 1.0081420	total: 302ms	remaining: 5m 1s
1:	learn: 0.9189432	total: 591ms	remaining: 4m 54s
2:	learn: 0.8587911	total: 922ms	remaining: 5m 6s
3:	learn: 0.8083211	total: 1.22s	remaining: 5m 3s
4:	learn: 0.7652208	total: 1.53s	remaining: 5m 4s
5:	learn: 0.7229399	total: 1.82s	remaining: 5m 1s
6:	learn: 0.6921498	total: 2.06s	remaining: 4m 52s
7:	learn: 0.6611266	total: 2.33s	remaining: 4m 48s
8:	learn: 0.6326218	total: 2.61s	remaining: 4m 47s
9:	learn: 0.6123134	total: 2.9s	remaining: 4m 47s
10:	learn: 0.5855651	total: 3.22s	remaining: 4m 49s
11:	learn: 0.5629631	total: 3.54s	remaining: 4m 51s
12:	learn: 0.5439233	total: 3.86s	remaining: 4m 53s
13:	learn: 0.5282106	total: 4.16s	remaining: 4m 52s
14:	learn: 0.5124537	total: 4.47s	remaining: 4m 53s
15:	learn: 0.4961844	total: 4.78s	remaining: 4m 54s
16:	learn: 0.4812680	total: 5s	remaining: 4m 48s
17:	learn: 0.4696736	total: 5.16s	remaining: 4m 41s
18:	learn: 0.4592623	total: 5.35s	remaining: 4m 36s
1

In [89]:
pred = clf.predict(features_test)
f1 = f1_scorrer(target_test, pred)
f1

0.6385034433138042

In [121]:
def transform_for_text(text, vectorizer, model):
  new_text = vectorizer.transform([text])
  pr = model.predict(text)
  return pr

In [91]:
test = write_all_news(f_all_news) #.drop(['Description', 'Publication Date'], axis=1) #все новости
test

,Title,Description,Links,Publication Date
0,Командующим войсками спецоперации на Украине н...,Командующим объединенной группировкой войск в ...,https://www.kommersant.ru/doc/5606182,"Sat, 08 Oct 2022 15:13:00 +0300"
1,Керченскую паромную переправу пообещали запуст...,"Оперативный штаб Краснодарского края сообщил, ...",https://www.kommersant.ru/doc/5606179,"Sat, 08 Oct 2022 15:04:00 +0300"
2,На месте паромной переправы в порту «Кавказ» о...,Вице-губернатор Краснодарского края Евгений На...,https://www.kommersant.ru/doc/5606175,"Sat, 08 Oct 2022 14:52:00 +0300"
3,В Воронежской области усилили охрану железнодо...,Власти Воронежской области по поручению губерн...,https://www.kommersant.ru/doc/5606174,"Sat, 08 Oct 2022 14:47:00 +0300"
4,Умер старейший депутат Сахалинской облдумы Вит...,На Сахалине в возрасте 80 лет умер старейший д...,https://www.kommersant.ru/doc/5606144,"Sat, 08 Oct 2022 14:25:00 +0300"
...,...,...,...,...
509,Россияне потеряли интерес к покупке жилья в по...,В сентябре 2022 года спрос со стороны россиян ...,https://lenta.ru/news/2022/10/07/no_florida/,"Fri, 07 Oct 2022 20:24:00 +0300"
510,Путин и Зеленский согласились приехать на самм...,Президент России Владимир Путин и украинский л...,https://lenta.ru/news/2022/10/07/g200/,"Fri, 07 Oct 2022 20:21:00 +0300"
511,Кремль связал слова Зеленского с проведением СВО,Слова президента Украины Владимира Зеленского ...,https://lenta.ru/news/2022/10/07/svoo/,"Fri, 07 Oct 2022 20:17:00 +0300"
512,В «Манчестер Юнайтед» оценили вероятность уход...,В английском клубе «Манчестер Юнайтед» оценили...,https://lenta.ru/news/2022/10/07/ronaldo/,"Fri, 07 Oct 2022 20:16:00 +0300"


In [137]:
def text_prepare(text):
  text = clear_text(text)
  txt = lemmatize1(text)
  return [txt]

In [135]:
def model_result(text, vectorizer, model):
  txt = text_prepare(text)
  vect_text = vectorizer.transform(txt)
  return model.predict(vect_text)[0][0]
#0 - бухгалтер
#1 - гендир
#2 - никто